In [15]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Create example data
# 1.1 Setup a single Bracewell

In [36]:
# combiner matrix for a single Bracewell array
combiner = np.array(((1, -1), (1, 1))) / np.sqrt(2)

# collector positions
baseline = 15  # in meter

# rotation angles over observation
rotation_angle = np.linspace(0., 2*np.pi, 1000)  # in rad

collector_positions_init = np.array(((-baseline/2, baseline/2), (0, 0)))

rotation_matrix = np.array(((np.cos(rotation_angle), -np.sin(rotation_angle)),
                            (np.sin(rotation_angle), np.cos(rotation_angle))))

collector_positions = np.dot(np.swapaxes(rotation_matrix, -1, 0), collector_positions_init)

# observing wavelength
wl_bin = 10e-6  # in meter

# collector area
scaled_area = 1  # in meter^2

## 1.2 Compute planet signal

In [48]:
# source position
separation_arcsec = 0.1  # in arcsec
separation_rad = separation_arcsec / 3600 / 180 * np.pi  # in rad

# rotation angles over observation
rotation_angle = 0.5  # in rad

# source position angle vectors in radians
source_position = np.array((separation_rad * np.cos(rotation_angle), 
                            separation_rad * np.sin(rotation_angle)))

# create input phasor
input_phasor = scaled_area * np.exp(1j * 2 * np.pi / wl_bin * np.dot(np.transpose(collector_positions, axes=(0, 2, 1)), source_position))

planet_signal = np.dot(combiner, input_phasor)

ValueError: shapes (2,2) and (1000,2) not aligned: 2 (dim 1) != 1000 (dim 0)

In [47]:
np.transpose(collector_positions, axes=(0, 2, 1)).shape

(1000, 2, 2)

In [49]:
source_position

array([4.25464032e-07, 2.32432060e-07])